In [2]:
import pandas as pd
import ast
import json
# from google.colab import files

# Upload the processed CSV file
uploaded = files.upload()  # Upload your file

# Load dataset
file_path = list(uploaded.keys())[0]  # Get the uploaded filename dynamically
df = pd.read_csv(file_path)

class ASTNode:
    def __init__(self, name, children=None):
        self.name = name
        self.children = children if children else []

    def add_child(self, child):
        self.children.append(child)

    def to_dict(self):
        return {
            "name": self.name,
            "children": [child.to_dict() for child in self.children]
        }

def build_ast_from_data(data):
    root = ASTNode("Root")

    for col in data.columns:
        col_node = ASTNode(col)
        root.add_child(col_node)

        for value in data[col].unique():
            value_node = ASTNode(str(value))
            col_node.add_child(value_node)

    return root

# Convert AST to JSON for structured output
def ast_to_json(ast_root):
    return json.dumps(ast_root.to_dict(), indent=4)

# Generate AST
ast_root = build_ast_from_data(df)
ast_json = ast_to_json(ast_root)

# Convert JSON to a DataFrame format
def ast_to_dataframe(ast_root, parent_name=""):
    rows = []
    node_name = f"{parent_name}/{ast_root.name}" if parent_name else ast_root.name
    rows.append({"Node": node_name, "Parent": parent_name if parent_name else None})

    for child in ast_root.children:
        rows.extend(ast_to_dataframe(child, node_name))

    return rows

# Convert AST to DataFrame
ast_df = pd.DataFrame(ast_to_dataframe(ast_root))

# Save the processed DataFrame to a CSV file
output_filename = "structured_ast.csv"
ast_df.to_csv(output_filename, index=False)

# Download the structured AST
files.download(output_filename)

print("AST stored successfully in structured_ast.csv")


NameError: name 'files' is not defined